# Distinguish Unitaries Kata Workbook

**What is this workbook?**
A workbook is a collection of problems, accompanied by solutions to them. 
The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required. 

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [Distinguish Unitaries kata](./DistinguishUnitaries.ipynb). Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a first-time user.

**What you should know for this workbook**

You should be familiar with the following concepts before tackling the Distinguish Unitaries kata (and this workbook):

1. [Basic linear algebra](./../tutorials/LinearAlgebra/LinearAlgebra.ipynb)).
2. [The concept of qubit](./../tutorials/Qubit/Qubit.ipynb) and [multi-qubit systems](./../tutorials/MultiQubitSystems/MultiQubitSystems.ipynb).
3. [Single-qubit](./../tutorials/SingleQubitGates/SingleQubitGates.ipynb) and [multi-qubit quantum gates](./../tutorials/MultiQubitGates/MultiQubitGates.ipynb) and using them to manipulate the state of the system.
4. Measurements and using them to distinguish quantum states.

To begin, first prepare this notebook for execution (if you skip the first step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells; if you skip the second step, you'll get "Invalid test name" error):

In [ ]:
%package Microsoft.Quantum.Katas::0.12.20082513

In [ ]:
%workspace reload

## Part I. Single-Qubit Gates

### Task 1.1. Identity or Pauli X?

**Input:** An operation that implements a single-qubit unitary transformation:
either the identity (**I** gate)
or the Pauli X gate (**X** gate). 
The operation will have Adjoint and Controlled variants defined.

**Output:**  0 if the given operation is the **I** gate, 1 if the given operation is the **X** gate.

You are allowed to apply the given operation and its adjoint/controlled variants exactly **once**.

### Solution

The only way to extract information out of a quantum system is measurement. 
Measurements give us information about the states of a system, so to get information about the gate, we need to find a way to convert it into information about a state.
If we want to distinguish two gates, we need to figure out to prepare a state and perform a measurement on it that will give us a result that we can interpret.
To do this, we’ll need to find a qubit state that, by applying to it I gate or X gate, will be transformed into states that can be distinguished using measurement, i.e., orthogonal states. 
Let's find such state.

> As a reminder, here are the matrices that correspond to the given gates:
> $$I = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}, X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$$

Consider the effects of these gates on the basis states: 

$$I|0\rangle = |0\rangle, I|1\rangle = |1\rangle$$
$$X|0\rangle = |1\rangle, X|1\rangle = |0\rangle$$

We see that the **I** gate leaves the $|0\rangle$ state unchanged, and the **X** gate transforms it into the $|1\rangle$ state. 
So the easiest thing to do is to prepare a $|0\rangle$ state, apply the given unitary to it and measure the resulting state in the computational basis:
* If the measurement result is `Zero`, the measured state was $|0\rangle$, and we know the unitary applied to it was the **I** gate.
* If the measurement result is `One`, the measured state was $|1\rangle$, and we know the unitary applied to it was the **X** gate.

> In Q#, the freshly allocated qubits start in the $|0\rangle$ state, so you don't need to do anything to prepare the necessary state before applying the unitary to it.
> You also have to return the qubits you allocated to the $|0\rangle$ state before releasing them. 
> You can do that by measuring the qubit using the `M` operation and applying the **X** gate if it was measured in the $|1\rangle$ state, or you can use [`MResetZ`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.measurement.mresetz) operation that wraps this measurement and fixup into one operation.

In [ ]:
%kata T101_DistinguishIfromX_Test 

operation DistinguishIfromX (unitary : (Qubit => Unit is Adj+Ctl)) : Int {
    using (q = Qubit()) {
        unitary(q);
        return M(q) == Zero ? 0 | 1;
    }
}

[Return to task 1.1 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-1.1.-Identity-or-Pauli-X?)

### Task 1.2. Identity or Pauli Z?

**Input:** An operation that implements a single-qubit unitary transformation:
either the identity (**I** gate)
or the Pauli Z gate (**Z** gate). 
The operation will have Adjoint and Controlled variants defined.

**Output:**  0 if the given operation is the **I** gate, 1 if the given operation is the **Z** gate.

You are allowed to apply the given operation and its adjoint/controlled variants exactly **once**.

### Solution

> As a reminder, $Z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}$

We won't be able to distinguish **I** from **Z** by applying them to the basis states, since they both leave the $|0\rangle$ state unchanged and add a phase to the $|1\rangle$ state: 

$$I|0\rangle = |0\rangle, I|1\rangle = |1\rangle$$
$$Z|0\rangle = |0\rangle, Z|1\rangle = -|1\rangle$$

However, if we try applying these gates to a superposition of basis states, we'll start seeing a difference between the resulting states:

$$I \big(\frac{1}{\sqrt2}(|0\rangle + |1\rangle)\big) = \frac{1}{\sqrt2}(|0\rangle + |1\rangle)$$
$$Z \big(\frac{1}{\sqrt2}(|0\rangle + |1\rangle)\big) = \frac{1}{\sqrt2}(|0\rangle \color{blue}{-} |1\rangle)$$

These two states are orthogonal and can be distinguished by measuring them in the $\{ |+\rangle, |-\rangle\}$ basis using [`MResetX`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.measurement.mresetx) operation (which is equivalent to applying an **H** gate and measuring in the computational basis).

> The task of distinguishing these two states is covered in more detail in the [Measurements kata](./..//Measurements/Measurements.ipynb#Task-1.3.-$|+\rangle$-or-$|-\rangle$?) and the corresponding workbook.

In [ ]:
%kata T102_DistinguishIfromZ_Test 

open Microsoft.Quantum.Measurement;

operation DistinguishIfromZ (unitary : (Qubit => Unit is Adj+Ctl)) : Int {
    using (q = Qubit()) {
        H(q);
        unitary(q);
        return MResetX(q) == Zero ? 0 | 1;
    }
}

[Return to task 1.2 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-1.2.-Identity-or-Pauli-Z?)

### Task 1.3. Z or S?

**Input:** An operation that implements a single-qubit unitary transformation:
either the **Z** gate
or the **S** gate. 
The operation will have Adjoint and Controlled variants defined.

**Output:**  0 if the given operation is the **Z** gate, 1 if the given operation is the **S** gate.

You are allowed to apply the given operation and its adjoint/controlled variants at most **twice**.

### Solution

> As a reminder, $S = \begin{bmatrix} 1 & 0 \\ 0 & i \end{bmatrix}$

This task differs from the previous two in that it allows you to apply the given unitary **twice**. 
Let's treat this as a hint that it is, and check how the given gates looks when applied twice. 
If you square the corresponding matrices (which is quite simple to do for diagonal matrices), you'll get

$$Z^2 = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix} = I, S^2 = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix} = Z$$

This means that the task of identifying the *square* of the given unitary transformation is the same as distinguishing **I** from **Z** gates - and that's exactly the [task 1.2](#Task-1.2.-Identity-or-Pauli-Z?)!

In [ ]:
%kata T103_DistinguishZfromS_Test 

open Microsoft.Quantum.Measurement;

operation DistinguishZfromS (unitary : (Qubit => Unit is Adj+Ctl)) : Int {
    using (q = Qubit()) {
        H(q);
        unitary(q);
        unitary(q);
        return MResetX(q) == Zero ? 0 | 1;
    }
}

[Return to task 1.3 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-1.3.-Z-or-S?)

### Task 1.4. Hadamard or X?

**Input:** An operation that implements a single-qubit unitary transformation:
either the Hadamard (**H**) gate
or the **X** gate. 
The operation will have Adjoint and Controlled variants defined.

**Output:**  0 if the given operation is the **H** gate, 1 if the given operation is the **X** gate.

You are allowed to apply the given operation and its adjoint/controlled variants at most **twice**.

### Solution

> As a reminder, $H = \frac{1}{\sqrt2} \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}$

The solution relies on the well-known identity $HXH = Z$ (i.e., Hadamard gate can be used to convert **X** to **Z** and vice versa). Applying a sequence of gates "given unitary - **X** - unitary" will be equivalent to $XXX = X$ gate if the unitary was **X**, and to $HXH = Z$ if the unitary was **H**. With this observation we need to distinguish **X** from **Z**, which can be done using the $|0\rangle$ state: **X** gate will change it to $|1\rangle$, and **Z** gate will leave it unchanged.

In [ ]:
%kata T104_DistinguishHfromX_Test 

operation DistinguishHfromX (unitary : (Qubit => Unit is Adj+Ctl)) : Int {
    using (q = Qubit()) {
        within {
            unitary(q);
        } apply {
            X(q);
        }
        return M(q) == Zero ? 0 | 1;
    }
}

[Return to task 1.4 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-1.4.-Hadamard-or-X?)

### Task 1.5. Z or $-$Z?

**Input:** An operation that implements a single-qubit unitary transformation:
either the **Z** gate
or the minus **Z** gate (i.e., the gate $- |0\rangle\langle0| + |1\rangle\langle1|$). 
The operation will have Adjoint and Controlled variants defined.

**Output:**  0 if the given operation is the **Z** gate, 1 if the given operation is the $-$**Z** gate.

You are allowed to apply the given operation and its adjoint/controlled variants exactly **once**.

### Solution

This task is more interesting: the given gates differ by a global phase they introduce (i.e., each of them is a multiple of the other), and the results of applying them to any single-qubit state are going to be indistinguishable by any measurement you can devise.

Fortunately, we are given not just the unitary itself, but also its controlled variant, i.e., the gate which applies the given unitary if the control qubit or qubits are in the $|1\rangle$ state, and does nothing if they are in the $|0\rangle$ state.
This allows us to use so called "phase kickback" trick, in which applying a controlled version of a gate allows us to observe the phase introduced by this gate on the control qubit. Indeed,

| State | Controlled Z | Controlled $-$Z |
|-------|---------------|------|
| $|00\rangle$ | $|00\rangle$ | $|00\rangle$ |
| $|01\rangle$ | $|01\rangle$ | $|01\rangle$ |
| $|10\rangle$ | $\color{blue}{|10\rangle}$  | $\color{blue}{-|10\rangle}$ |
| $|11\rangle$ | $\color{blue}{-|11\rangle}$ | $\color{blue}{|11\rangle}$  |

We see that both controlled gates don't modify the states with the control qubit in the $|0\rangle$ state, but if the control qubit is in the $|1\rangle$ state, they introduce a $-1$ phase to different basis states. 
We can take advantage of this if we apply the controlled gate to a state in which the *control qubit* is in superposition, such as $\frac{1}{\sqrt2}(|0\rangle + |1\rangle) \otimes |0\rangle$:

$$\text{Controlled Z}\frac{1}{\sqrt2}(|0\rangle + |1\rangle) \otimes |0\rangle = \frac{1}{\sqrt2}(|0\rangle + |1\rangle) \otimes |0\rangle$$
$$\text{Controlled }-\text{Z}\frac{1}{\sqrt2}(|0\rangle + |1\rangle) \otimes |0\rangle = \frac{1}{\sqrt2}(|0\rangle - |1\rangle) \otimes |0\rangle$$

After this we can measure the first qubit to distinguish $\frac{1}{\sqrt2}(|0\rangle + |1\rangle)$ from $\frac{1}{\sqrt2}(|0\rangle - |1\rangle)$, like we did in [task 1.2](#Task-1.2.-Identity-or-Pauli-Z?).

> In Q# we can express controlled version of a gate using [Controlled functor](https://docs.microsoft.com/en-us/quantum/user-guide/using-qsharp/operations-functions#controlled-functor): the first argument of the resulting gate will be an array of control qubits, and the second one - the arguments of the original gate (in this case just the target qubit).

In [ ]:
%kata T105_DistinguishZfromMinusZ_Test 

open Microsoft.Quantum.Measurement;

operation DistinguishZfromMinusZ (unitary : (Qubit => Unit is Adj+Ctl)) : Int {
    using (qs = Qubit[2]) {
        // prep (|0⟩ + |1⟩) ⊗ |0⟩
        H(qs[0]);

        Controlled unitary(qs[0..0], qs[1]);

        return MResetX(qs[0]) == Zero ? 0 | 1;
    }
}

[Return to task 1.5 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-1.5.-Z-or-$-$Z?)

### Task 1.6. Rz or R1 (arbitrary angle)?

**Input:** An operation that implements a single-qubit unitary transformation:
either the [**Rz** gate](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.rz)
or the [**R1** gate](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.r1). 

This operation will take two parameters: the first parameter is the rotation angle, in radians, and the second parameter is the qubit to which the gate should be applied (matching normal **Rz** and **R1** gates in Q#).
The operation will have Adjoint and Controlled variants defined.

**Output:**  0 if the given operation is the **Rz** gate, 1 if the given operation is the **R1** gate.

You are allowed to apply the given operation and its adjoint/controlled variants exactly **once**.

### Solution

> As a reminder, $$R_z(\theta) = \begin{bmatrix} e^{-i\theta/2} & 0 \\ 0 & e^{i\theta/2} \end{bmatrix} \text{, } 
R_1(\theta) = \begin{bmatrix} 1 & 0 \\ 0 & e^{i\theta} \end{bmatrix} = e^{i\theta/2} R_z(\theta)$$

We see that these two gates differ by a global phase $e^{i\theta/2}$.
In this problem we're free to choose the angle parameter which we'll pass to our gate, so we can choose an angle that make this global phase difference something easy to detect: for $\theta = 2\pi$ $e^{i\theta/2} = -1$, so $R_z(\theta) = -I$, and $R_1(\theta) = I$.

Now we need to distinguish **I** gate from $-$**I** gate, which can be done using controlled variant of the gate in exactly the same way as distinguishing **Z** gate from $-$**Z** gate in [task 1.6](#Task-1.5.-Z-or-$-$Z?).

In [ ]:
%kata T106_DistinguishRzFromR1_Test 

open Microsoft.Quantum.Math;

operation DistinguishRzFromR1 (unitary : ((Double, Qubit) => Unit is Adj+Ctl)) : Int {
    using (qs = Qubit[2]) {
        within {
            H(qs[0]);
        } apply {
            Controlled unitary(qs[0..0], (2.0 * PI(), qs[1]));
        }
        return M(qs[0]) == Zero ? 1 | 0;
    }
}

[Return to task 1.6 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-1.6.-Rz-or-R1-(arbitrary-angle)?)

### Task 1.7. Y or XZ?

**Input:** An operation that implements a single-qubit unitary transformation:
either the **Y** gate
or the sequence of Pauli **Z** and Pauli **X** gates (equivalent to applying the **Z** gate followed by the **X** gate).
The operation will have Adjoint and Controlled variants defined.

**Output:**  0 if the given operation is the **Y** gate, 1 if the given operation is the **XZ** gate.

You are allowed to apply the given operation and its adjoint/controlled variants at most **twice**.

### Solution

> As a reminder, $$Y = \begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix} = iXZ$$

We see that these two gates differ by a global phase $i = e^{i\pi}$. Applying the gates twice will give us gates $Y^2 = I$ and $(XZ)^2 = XZXZ = -XXZZ = -I$, respectively.

Now we need to distinguish **I** gate from $-$**I** gate, which is the same thing we did in the previous task.

In [ ]:
%kata T107_DistinguishYfromXZ_Test 

operation DistinguishYfromXZ (unitary : (Qubit => Unit is Adj+Ctl)) : Int {
    using (qs = Qubit[2]) {
        // prep (|0⟩ + |1⟩) ⊗ |0⟩
        within { H(qs[0]); }
        apply {  
            Controlled unitary(qs[0..0], qs[1]);
            Controlled unitary(qs[0..0], qs[1]);
        }

        // 0 means it was Y
        return M(qs[0]) == Zero ? 0 | 1;
    }
}

[Return to task 1.7 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-1.7.-Y-or-XZ?)

### Task 1.8. Y, XZ, $-$Y or $-$XZ?

**Input:** An operation that implements a single-qubit unitary transformation:
either the **Y** gate (possibly with an extra global phase of $-1$) or the sequence of Pauli **Z** and Pauli **X** gates (possibly with an extra global phase of $-1$).
The operation will have Adjoint and Controlled variants defined.

**Output:** 
* 0 if the given operation is the **Y** gate,
* 1 if the given operation is the $-$**XZ** gate,
* 2 if the given operation is the $-$**Y** gate,
* 3 if the given operation is the **XZ** gate.

You are allowed to apply the given operation and its adjoint/controlled variants at most **three times**.

### Solution 1

In this task we have to distinguish 4 gates that were identical up to a global phase, i.e., gates $Y$, $iY$, $-Y$ and $-iY$ (in that order). 

One way to do this is "by hand", similar to the previous tasks. First we'll apply the controlled variant of the unitary twice and check whether the result is $I$ or $-I$ (which allows us to distinguish $\pm Y$ and $\pm iY$, since the first two gates squared will be equivalent to the $I$ gate, and the last two &mdash; to the $-I$ gate). Then we distinguish the gates in each group ($Y$ from $-Y$ or $iY$ from $-iY$) by applying the controlled variant of the unitary once.

In [ ]:
%kata T108_DistinguishYfromXZWithPhases_Test 

open Microsoft.Quantum.Measurement;

operation DistinguishYfromXZWithPhases (unitary : (Qubit => Unit is Adj+Ctl)) : Int {
    using ((control, target) = (Qubit(), Qubit())) {
        // Distinguish Y from iY
        within {
            H(control);
        } apply {
            Controlled unitary([control], target);
            Controlled unitary([control], target);
        }
        Reset(target);
        let isY = MResetZ(control) == Zero;
        
        // Distinguish Y from -Y and iY from -iY
        within {
            H(control);
        } apply {
            Controlled unitary([control], target);
            // apply controlled variant of the gate we're expecting to compensate effect on target qubit
            if (isY) {
                CY(control, target);
            } else {
                CZ(control, target);
                CX(control, target);
            }
        }
        
        return (isY ? M(control) == Zero ? 0 | 2
                    | M(control) == Zero ? 1 | 3);
    }
}

### Solution 2

An alternate solution takes advantage of [phase estimation](https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm) algorithm which, given a unitary and its eigenstate, allows to find its eigenvalue. If we prepare an eigenstate of the Y gate $\frac{1}{\sqrt2}(|0\rangle + i|1\rangle)$, it will be the eigenstate of all other gates as well, with eigenphases $0, \pi/2, \pi, 3\pi/2$, respectively. You can convert the output of the phase estimation procedure into the return index directly.

You can learn more about phase estimation in [this kata](../PhaseEstimation/PhaseEstimation.ipynb).

In [ ]:
operation OraclePowerWrapper (U : (Qubit => Unit is Adj + Ctl), power : Int, target : Qubit[]) : Unit is Adj + Ctl {
    for (_ in 1 .. power) {
        U(target[0]);
    }
}

In [ ]:
%kata T108_DistinguishYfromXZWithPhases_Test 

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Characterization;
open Microsoft.Quantum.Oracles;

operation DistinguishYfromXZWithPhases (unitary : (Qubit => Unit is Adj+Ctl)) : Int {
    // Run phase estimation on the unitary and the +1 eigenstate of the Y gate |0⟩ + i|1⟩

    // Construct a phase estimation oracle from the unitary
    let oracle = DiscreteOracle(OraclePowerWrapper(unitary, _, _));

    // Allocate qubits to hold the eigenstate of U and the phase in a big endian register 
    mutable phaseInt = 0;
    using ((eigenstate, phaseRegister) = (Qubit[1], Qubit[2])) {
        let phaseRegisterBE = BigEndian(phaseRegister);
        // Prepare the eigenstate of U
        H(eigenstate[0]); 
        S(eigenstate[0]);
        // Call library
        QuantumPhaseEstimation(oracle, eigenstate, phaseRegisterBE);
        // Read out the phase
        set phaseInt = MeasureInteger(BigEndianAsLittleEndian(phaseRegisterBE));

        ResetAll(eigenstate);
        ResetAll(phaseRegister);
    }

    // Convert the measured phase into return value
    return phaseInt;
}

[Return to task 1.8 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-1.8.-Y,-XZ,-$-$Y-or-$-$XZ?)

### Task 1.9. Rz or Ry (fixed angle)?

**Inputs:** 

1. An angle $\theta \in [0.01 \pi; 0.99 \pi]$.
2. An operation that implements a single-qubit unitary transformation:
either the [$R_z(\theta)$ gate](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.rz)
or the [$R_y(\theta)$ gate](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.ry). 

The operation will have Adjoint and Controlled variants defined.

**Output:**  0 if the given operation is the **Rz** gate, 1 if the given operation is the **Ry** gate.

You are allowed to apply the given operation and its adjoint/controlled variants **any number of times**.

### Solution

> As a reminder,
> 
> $$R_z(\theta) = \begin{bmatrix} e^{-i\theta/2} & 0 \\ 0 & e^{i\theta/2} \end{bmatrix} \\
R_y(\theta) = \begin{bmatrix} \cos\frac{\theta}{2} & -\sin\frac{\theta}{2} \\ \sin\frac{\theta}{2} & \cos\frac{\theta}{2} \end{bmatrix}$$

The key observation here is that $R_z$ is a diagonal matrix and $R_y$ is not, so when applied to the $|0\rangle$ state, the former will leave it unchanged (with an extra phase which is not observable), and the latter will convert it to a superposition $\cos\frac{\theta}{2} |0\rangle + \sin\frac{\theta}{2} |1\rangle$. The question is, how to distinguish those two states if they are not orthogonal (and for most values of $\theta$ they will not be)?

The task description gives you a big hint: it allows you to use the given unitary unlimited number of times, which points to a probabilistic solution (as opposed to deterministic solutions in all previous problems in this kata). Apply the unitary to the $|0\rangle$ state and measure the result; if it is $|1\rangle$, the unitary must be $R_y$, otherwise you can repeat the experiment again. After several iterations of measuring $|0\rangle$ you can conclude that with high probability the unitary is $R_z$.

To reduce the number of iterations after which you make the decision, you could apply the unitary several times to bring the overall rotation angle closer to $\pi$: in case of $R_y$ gate this would allow you to rotate the state closer to the $|1\rangle$ state, so that you'd detect it with higher probability.

In [ ]:
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;

function ComputeRepetitions(angle : Double, offset : Int, accuracy : Double) : Int {
    mutable pifactor = 0;
    while (true) {
        let pimultiple = PI() * IntAsDouble(2 * pifactor + offset);
        let times = Round(pimultiple / angle);
        if (AbsD(pimultiple - (IntAsDouble(times) * angle)) / PI() < accuracy) {
            return times;
        }
        set pifactor += 1;
    }
    return 0;
}

In [ ]:
%kata T109_DistinguishRzFromRy_Test 

open Microsoft.Quantum.Measurement;

operation DistinguishRzFromRy (theta : Double, unitary : (Qubit => Unit is Adj+Ctl)) : Int {
    using (q = Qubit()) {
        let times = ComputeRepetitions(theta, 1, 0.1);
        mutable attempt = 1;
        mutable measuredOne = false;
        repeat {
            for (_ in 1..times) {
                unitary(q);
            }
            // for Rz, we'll never venture away from |0⟩ state, so as soon as we got |1⟩ we know it's not Rz
            if (MResetZ(q) == One) {
                set measuredOne = true;
            }
            // if we try several times and still only get |0⟩s, chances are that it is Rz
        } until (attempt == 4 or measuredOne) 
        fixup {
            set attempt += 1;
        }
        return measuredOne ? 1 | 0;
    }
}

[Return to task 1.9 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-1.9.-Rz-or-Ry-(fixed-angle)?)

### Task 1.11. Distinguish 4 Pauli unitaries

**Input:** An operation that implements a single-qubit unitary transformation:
either the identity (**I** gate) or one of the Pauli gates (**X**, **Y** or **Z** gate).
The operation will have Adjoint and Controlled variants defined.

**Output:** 
* 0 if the given operation is the **I** gate,
* 1 if the given operation is the **X** gate,
* 2 if the given operation is the **Y** gate,
* 3 if the given operation is the **Z** gate.

You are allowed to apply the given operation and its adjoint/controlled variants exactly **once**.

### Solution

This task is quite different from the previous tasks in this section; 
at the first glance it might seem impossible to distinguish four different unitaries (i.e., get two bits of information) with just one unitary application! 

However, since the unitaries were chosen carefully (and you're not limited in the number of measurements you can do), it is possible. 
The solution uses the [Bell states](../Superposition/Superposition.ipynb#all-bell-states): the 4 orthogonal states which you can prepare by starting with the first of them $\frac{1}{\sqrt2}(|00\rangle + |11\rangle)$ and applying the gates **I**, **X**, **Z** and **Y**, respectively, to the first qubit. 
Thus the solution becomes: prepare the $\frac{1}{\sqrt2}(|00\rangle + |11\rangle)$ state, apply the unitary and measure the resulting state in Bell basis to figure out which of the Bell states it is 
(see the [Measurements kata](../Measurements/Measurements.ipynb#Task-1.12.-Distinguish-four-Bell-states.) for the details on how to do that).

In [ ]:
%kata T111_DistinguishPaulis_Test 

operation DistinguishPaulis (unitary : (Qubit => Unit is Adj+Ctl)) : Int {
    // apply operation to the 1st qubit of a Bell state and measure in Bell basis
    using (qs = Qubit[2]) {
        within {
            H(qs[0]);
            CNOT(qs[0], qs[1]);
        } apply {
            unitary(qs[0]);
        }

        // after this I -> 00, X -> 01, Y -> 11, Z -> 10
        let ind = MeasureInteger(LittleEndian(qs));
        let returnValues = [0, 3, 1, 2];
        return returnValues[ind];
    }
}

[Return to task 1.11 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-1.11.-Distinguish-4-Pauli-unitaries)

## Part II. Multi-Qubit Gates

### Task 2.1. $I \otimes X$ or $\text{CNOT}$?

**Input:** An operation that implements a two-qubit unitary transformation:
either the $I \otimes X$ (the **X** gate applied to the second qubit)
or the $\text{CNOT}$ gate with the first qubit as control and the second qubit as target.
* The operation will accept an array of qubits as input, but it will fail if the array is empty or has one or more than two qubits.
* The operation will have Adjoint and Controlled variants defined.

**Output:**  0 if the given operation is $I \otimes X$, 1 if the given operation is the $\text{CNOT}$ gate.

You are allowed to apply the given operation and its adjoint/controlled variants exactly **once**.

### Solution

Let's consider the effect of these gates on the basis states:

| State | $I \otimes X$ | **CNOT** |
|-------|---------------|------|
| $|00\rangle$ | $|01\rangle$ | $|00\rangle$ |
| $|01\rangle$ | $|00\rangle$ | $|01\rangle$ |
| $|10\rangle$ | $|11\rangle$ | $|11\rangle$ |
| $|11\rangle$ | $|10\rangle$ | $|10\rangle$ |

We can see that applying these two gates to states with the first qubit in the $|1\rangle$ state yields identical results, but applying them to states with the first qubit in the $|0\rangle$ state produces states that differ in the second qubuit.
This makes sense, since the **CNOT** gate is defined as "apply **X** gate to the target qubit if the control qubit is in the $|1\rangle$ state, and do nothing if it is in the $|0\rangle$ state".

Thus, the easiest solution is: allocate two qubits in the $|00\rangle$ state and apply the unitary to them, then measure the second qubit; if it is `One`, the gate is $I \otimes X$, otherwise it's **CNOT**.

In [ ]:
%kata T201_DistinguishIXfromCNOT_Test

open Microsoft.Quantum.Measurement;

operation DistinguishIXfromCNOT (unitary : (Qubit[] => Unit is Adj+Ctl)) : Int {
    using (qs = Qubit[2]) {
        unitary(qs);
        return MResetZ(qs[1]) == One ? 0 | 1;
    }
}

[Return to task 2.1 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-2.1.-$I-\otimes-X$-or-$\text{CNOT}$?)

### Task 2.2. Figure out the direction of $\text{CNOT}$

**Input:** An operation that implements a two-qubit unitary transformation:
either the $\text{CNOT}$ gate with the first qubit as control and the second qubit as target ($\text{CNOT}_{12}$)
or the $\text{CNOT}$ gate with the second qubit as control and the first qubit as target ($\text{CNOT}_{21}$).
* The operation will accept an array of qubits as input, but it will fail if the array is empty or has one or more than two qubits.
* The operation will have Adjoint and Controlled variants defined.

**Output:**  0 if the given operation is $\text{CNOT}_{12}$, 1 if the given operation is $\text{CNOT}_{21}$.

You are allowed to apply the given operation and its adjoint/controlled variants exactly **once**.

### Solution

Again, let's consider the effect of these gates on the basis states:

| State | $\text{CNOT}_{12}$ | $\text{CNOT}_{21}$ |
|-------|---------------|------|
| $|00\rangle$ | $|00\rangle$ | $|00\rangle$ |
| $|01\rangle$ | $|01\rangle$ | $|11\rangle$ |
| $|10\rangle$ | $|11\rangle$ | $|10\rangle$ |
| $|11\rangle$ | $|10\rangle$ | $|01\rangle$ |

We can see that applying these two gates to any basis state other than $|00\rangle$ yields different results, and we can use any of these states to distinguish the unitaries.

Thus, the easiest solution is: prepare two qubits in the $|01\rangle$ state and apply the unitary to them, then measure the first qubit; if it is still `Zero`, the gate is $\text{CNOT}_{12}$, otherwise it's $\text{CNOT}_{21}$.

In [ ]:
%kata T202_CNOTDirection_Test

operation CNOTDirection (unitary : (Qubit[] => Unit is Adj+Ctl)) : Int {
    using (qs = Qubit[2]) {
        within { X(qs[1]); }
        apply { unitary(qs); }
        return M(qs[0]) == Zero ? 0 | 1;
    }
}

[Return to task 2.2 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-2.2.-Figure-out-the-direction-of-$\text{CNOT}$)

### Task 2.3. $\text{CNOT}_{12}$ or $\text{SWAP}$?

**Input:** An operation that implements a two-qubit unitary transformation:
either the $\text{CNOT}$ gate with the first qubit as control and the second qubit as target ($\text{CNOT}_{12}$)
or the $SWAP$ gate.
* The operation will accept an array of qubits as input, but it will fail if the array is empty or has one or more than two qubits.
* The operation will have Adjoint and Controlled variants defined.

**Output:**  0 if the given operation is $\text{CNOT}_{12}$, 1 if the given operation is $\text{SWAP}$.

You are allowed to apply the given operation and its adjoint/controlled variants exactly **once**.

### Solution

Again, let's consider the effect of these gates on the basis states:

| State | $\text{CNOT}_{12}$ | $\text{SWAP}$ |
|-------|---------------|------|
| $|00\rangle$ | $|00\rangle$ | $|00\rangle$ |
| $|01\rangle$ | $|01\rangle$ | $|10\rangle$ |
| $|10\rangle$ | $|11\rangle$ | $|01\rangle$ |
| $|11\rangle$ | $|10\rangle$ | $|11\rangle$ |

Same as in the previous task, applying these two gates to any basis state other than $|00\rangle$ yields different results, and we can use any of these states to distinguish the unitaries.

The easiest solution is: prepare two qubits in the $|01\rangle$ state and apply the unitary to them, then measure the first qubit; if it is still `Zero`, the gate is $\text{CNOT}_{12}$, otherwise it's $\text{SWAP}$. Remember that this time the second qubit might end up in $|1\rangle$ state, so it needs to be reset to $|0\rangle$ before releasing it.

In [ ]:
%kata T203_DistinguishCNOTfromSWAP_Test

operation DistinguishCNOTfromSWAP (unitary : (Qubit[] => Unit is Adj+Ctl)) : Int {
    using (qs = Qubit[2]) {
        X(qs[1]);
        unitary(qs);
        Reset(qs[1]);
        return M(qs[0]) == Zero ? 0 | 1;
    }
}

[Return to task 2.3 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-2.3.-$\text{CNOT}_{12}$-or-$\text{SWAP}$?)

### Task 2.4. Identity, $\text{CNOT}$s or $\text{SWAP}$?

**Input:** An operation that implements a two-qubit unitary transformation:
either the identity ($I \otimes I$), the $\text{CNOT}$ gate with one of the qubits as control and the other qubit as a target, 
or the $SWAP$ gate.
* The operation will accept an array of qubits as input, but it will fail if the array is empty or has one or more than two qubits.
* The operation will have Adjoint and Controlled variants defined.

**Output:**  

* 0 if the given operation is $I \otimes I$, 
* 1 if the given operation is $\text{CNOT}_{12}$,
* 2 if the given operation is $\text{CNOT}_{21}$,
* 3 if the given operation is $\text{SWAP}$.

You are allowed to apply the given operation and its adjoint/controlled variants at most **twice**.

### Solution

In this problem we are allowed to use the given unitary twice, so we can split our decision-making process in two phases:

1. Apply the unitary to the $|11\rangle$ state; $\text{CNOT}_{12}$ will yield the $|10\rangle$ state, $\text{CNOT}_{21}$ &mdash; $|01\rangle$, and both $I \otimes I$ and $\text{SWAP}$ gates will leave the state unchanged.
2. Now to distinguish $I \otimes I$ from $\text{SWAP}$, we can use the $|01\rangle$ state: $I \otimes I$ gate will leave it unchanged, while $\text{SWAP}$ will yield $|10\rangle$.

In [ ]:
%kata T204_DistinguishTwoQubitUnitaries_Test

open Microsoft.Quantum.Arithmetic;

operation DistinguishTwoQubitUnitaries (unitary : (Qubit[] => Unit is Adj+Ctl)) : Int {
    // first run: apply to |11⟩; CNOT₁₂ will give |10⟩, CNOT₂₁ will give |01⟩, II and SWAP will remain |11⟩
    using (qs = Qubit[2]) {
        ApplyToEach(X, qs);
        unitary(qs);
        let ind = MeasureInteger(LittleEndian(qs));
        if (ind == 1 or ind == 2) {
            // respective CNOT
            return ind;
        }
    }
    // second run: distinguish II from SWAP, apply to |01⟩: II will remain |01⟩, SWAP will become |10⟩
    using (qs = Qubit[2]) {
        X(qs[1]);
        unitary(qs);
        let ind = MeasureInteger(LittleEndian(qs));
        return ind == 1 ? 3 | 0;
    }
}

[Return to task 2.4 of the Distinguish Unitaries kata.](./DistinguishUnitaries.ipynb#Task-2.4.-Identity,-$\text{CNOT}$s-or-$\text{SWAP}$?)